In [1]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta, time
import nest_asyncio
nest_asyncio.apply()  # Allow nested async loops in Jupyter

# Main coroutine
async def fetch_ticks_for_range():
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=15)

    contract = Stock('GOOG', 'SMART', 'USD')

    D1 = datetime(2025, 7, 7)
    D2 = datetime(2025, 7, 8)
    trading_days = pd.date_range(D1, D2, freq='B')  # Business days

    all_dfs = []

    for day in trading_days:
        start_dt = datetime.combine(day.date(), time(15, 50))
        end_dt = datetime.combine(day.date(), time(16, 0))

        try:
            print(f"📡 Fetching for {day.date()}...")

            ticks = ib.reqHistoricalTicks(
                contract,
                startDateTime=start_dt,
                endDateTime=end_dt,
                numberOfTicks=1000,
                whatToShow='TRADES',
                useRth=True,
                ignoreSize=True
            )

            # Convert to DataFrame
            df = util.df(ticks)
            df['date'] = day.date()
            all_dfs.append(df)

        except Exception as e:
            print(f"❌ Error on {day.date()}: {e}")

    ib.disconnect()

    if all_dfs:
        full_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\n✅ Total ticks fetched: {len(full_df)}")
        display(full_df.head())
        return full_df
    else:
        print("⚠️ No tick data fetched")
        return None

# Run the coroutine inside Jupyter
full_df = await fetch_ticks_for_range()


📡 Fetching for 2025-07-07...
📡 Fetching for 2025-07-08...

✅ Total ticks fetched: 2365


,time,tickAttribLast,price,size,exchange,specialConditions,date
0,2025-07-07 19:50:00+00:00,"TickAttribLast(pastLimit=False, unreported=True)",176.97,23.0,BATS,F I,2025-07-07
1,2025-07-07 19:50:00+00:00,"TickAttribLast(pastLimit=False, unreported=True)",176.97,17.0,BATS,F I,2025-07-07
2,2025-07-07 19:50:00+00:00,"TickAttribLast(pastLimit=False, unreported=True)",176.97,4.0,BATS,F I,2025-07-07
3,2025-07-07 19:50:00+00:00,"TickAttribLast(pastLimit=False, unreported=True)",176.97,19.0,BATS,F I,2025-07-07
4,2025-07-07 19:50:00+00:00,"TickAttribLast(pastLimit=False, unreported=False)",176.97,100.0,BATS,F,2025-07-07
